In [1]:
import pandas as pd

In [2]:
#get tables from wikipedia
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df.sample(n=5, random_state=1)

,Postal Code,Borough,Neighbourhood
116,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
112,M5R,Central Toronto,"The Annex, North Midtown, Yorkville"
99,M1P,Scarborough,"Dorset Park, Wexford Heights, Scarborough Town..."
161,M9X,Not assigned,Not assigned
35,M9E,Not assigned,Not assigned


In [3]:
#drop not assigned borough
df = df[df.Borough != 'Not assigned']
df.sample(n=5, random_state=1)

,Postal Code,Borough,Neighbourhood
94,M5N,Central Toronto,Roselawn
65,M3K,North York,Downsview
64,M2K,North York,Bayview Village
104,M6P,West Toronto,"High Park, The Junction South"
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [4]:
#replace not assigned neighbourhood with borough
df[df.Neighbourhood == 'Not assigned']
#welp that's easy lol, none of them are

,Postal Code,Borough,Neighbourhood


##all borough with NA values is within all neighbourhood with NA values

In [5]:
df.shape

(103, 3)